# Test on VGG11 Tensorflow

The dataset is trained off-line. Each client has 1 sample from every class, hence, 10 condensed samples per clients. we considered 10 clients in this scenario. the condensed data were trained based on ConvNet architecture. Here we train VGG11 in a federated learning paradigm as a cross-architecture test scinario.

In [ ]:
# ! git clone https://github.com/VICO-UoE/DatasetCondensation.git
# %cd DatasetCondensation/
%cd /content/drive/MyDrive/Project B/DatasetCondensation-master
import sys
import torch
sys.path.append('/content/drive/MyDrive/Project B/DatasetCondensation-master')
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import scipy.io as so
import matplotlib.pyplot as plt
import numpy as np
import argparse
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug


/content/drive/MyDrive/Project B/DatasetCondensation-master


In [ ]:
import tensorflow as tf
class Block(tf.keras.models.Sequential):
    def __init__(self,n,m):
        super().__init__()
        for i in range(m):
            self.add(tf.keras.layers.Conv2D(filters = n, kernel_size=(3,3),strides=(1,1),padding = 'same',activation = "relu"))
        self.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2)))

class Dense(tf.keras.models.Sequential):
    def __init__(self,n,m=2):
        super().__init__()
        for i in range(m):
            self.add(tf.keras.layers.Dense(units = n, activation = "relu"))
class VGG11(tf.keras.models.Sequential):
    def __init__(self, input_shape, classes, filters = 64):
        super().__init__()
        self.add(tf.keras.layers.InputLayer(input_shape = input_shape))

        # Backbone
        self.add(Block(n = filters * 1, m = 1))
        self.add(Block(n = filters * 2, m = 1))
        self.add(Block(n = filters * 4, m = 2))
        self.add(Block(n = filters * 8, m = 2))
        self.add(Block(n = filters * 8, m = 2))

        # top
        self.add(tf.keras.layers.Flatten())
        self.add(Dense(n = filters * 64))
        self.add(tf.keras.layers.Dense(units = classes,activation = "softmax"))

In [ ]:
im_syn = so.loadmat('/content/drive/MyDrive/Project B/DatasetCondensation-master/result_for_FL/image_syn.m')
im_syn = im_syn['image_syn']
label_syn = so.loadmat('/content/drive/MyDrive/Project B/DatasetCondensation-master/result_for_FL/label_syn.m')
label_syn = label_syn['label_syn']
_, _, _, _, _, _, _, dst_test, _ = get_dataset('CIFAR10', 'data')

In [ ]:

test_imgs = np.zeros((10000,3,32,32))
test_labels = np.zeros(10000)
for i in range(10000):
  test_imgs[i,...] = dst_test.__getitem__(i)[0]
  test_labels[i,...] = dst_test.__getitem__(i)[1]

In [ ]:
def transform_images (imgs):
  ims = np.zeros((imgs.shape[0],imgs.shape[2],imgs.shape[3],imgs.shape[1]))
  for i in range(imgs.shape[0]):
    for ch in range(3):
     ims[i,...,ch] = np.fliplr (imgs[i,ch,...])
  return ims

In [ ]:
imgs_train = transform_images(im_syn)
imgs_test = transform_images(test_imgs)

In [ ]:
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append( list() ) #different object reference each time
    return list_of_objects

In [ ]:
dataset_train = init_list_of_objects(10)
dataset_label = init_list_of_objects(10)

In [ ]:
num_clients = 10
temp1 = np.zeros((10,32,32,3))
temp2 = np.zeros(10)
for client in range(num_clients):
  for i in range (10):
    temp1[i,...] = imgs_train[10*i+client,...]
    temp2[i] = label_syn[0,10*i+client]
  dataset_train[client].append( temp1)
  dataset_label[client].append(temp2)

In [ ]:
# plt.imshow(dataset_train[4][0][1,...])

In [ ]:
def evaluate(client_weights,im,label,client_id):
  keras_model = VGG11((32,32,3),10)
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(client_weights)
  client_xdata = im
  client_ydata = label
  return keras_model.evaluate(client_xdata,client_ydata)

In [ ]:
import random
acc1 = np.zeros((10000))
rounds = 2000
E = 6
N_active_clients = 10
N_total_clients = 10
Batch_size = 10
client_list = range(N_total_clients)
client_list = sorted(client_list, key = lambda x: random.random() )

model = VGG11((32,32,3),10)
optimizer_clients = tf.keras.optimizers.SGD(learning_rate=0.02)  
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)                                                     
model.compile(loss=loss,
              optimizer=optimizer_clients,
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()] )                                                          

global_var = model.get_weights()

client_data_size = 10
total_data_size = 100

In [ ]:

############## Federated Learning  ####################################################
for round in range(rounds):
  client_list = np.random.randint(0,N_total_clients,N_active_clients)
  client_vars_sum = None
  print("round ",round)

  
  
  for client in client_list:
    print("Client No ",client)
    client_id = np.random.randint(0,N_total_clients)
    client_data_size = dataset_train[client_id][0].shape[0]
    client_xdata = dataset_train[client_id][0]
    client_ydata = dataset_label[client_id][0]

    model = VGG11((32,32,3),10)
    optimizer_clients = tf.keras.optimizers.SGD(learning_rate=0.02)  
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)                                                     
    model.compile(loss=loss,
              optimizer=optimizer_clients,
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()] )

    model.set_weights(global_var)
    
    for epoch in range(E):
      
      # idx1 = np.random.randint(0, client_data, size=mixed_t.shape[0])
    #  mixed_batch_t, sig_1_batch_t= mixed_t[idx1], sig_1_t[idx1], 
      
      info = model.fit(client_xdata, client_ydata ,batch_size=Batch_size,
  epochs=1,
  verbose=1,
  callbacks=None,
  validation_split=0.0,
  validation_data=None,
  shuffle=True,)
    
    client_var = model.get_weights()
    sum_list =[]
    if client_vars_sum is None:
      # weight = data_size / total_data_size
      weight = 1/N_active_clients
      client_vars_sum = [weight * x for x in client_var]
    else:
      for (item1, item2) in zip(client_var, client_vars_sum):
        sum_list.append( weight*item1 + item2)

      client_vars_sum = sum_list
  
  global_var = client_vars_sum
  print (50*"#" +" Evaluate on test dataset (10000 CIFAR10 test samples) "+ 50*"#")
  accuracy = evaluate(global_var,imgs_test,test_labels,0)
  acc1[round] = accuracy[1]
  
  print("The test accuracy for ALL clients  is "+str(accuracy[1]))
  if rounds % 5==0:
    plt.plot(acc1[:round])
    plt.xlabel('Epoch')
    plt.ylabel('Test accuracy')
    # plt.plot(acc[:round])
    plt.savefig("FL_accuracyVsepoch.jpg", bbox_inches='tight', dpi=200)
    plt.close()
    # if rounds % 20==0 and epoch>50:
        
        #  tf.keras.models.save_model(unet.generator1D,'/content/2_unet_generator_avgpooling_gen1k5_8filt_gen2k5_8filtk5_%diteration.h5' %epoch)




round  0
Client No  7
1/1 [==============================] - 0s 499ms/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  4
1/1 [==============================] - 1s 937ms/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  0
1/1 [==============================] - 2s 2s/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  5
1/1 [==============================] - 1s 912ms/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  3
1/1 [==============================] - 2s 2s/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  8
1/1 [==============================] - 1s 1s/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  9
1/1 [==============================] - 1s 944ms/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  3
1/1 [==============================] - 1s 950ms/step - loss: 2.2812 - sparse_categorical_accuracy: 0.5000
Client No  3
1/1 [==============================

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


round  1
Client No  8
1/1 [==============================] - 0s 469ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  3
1/1 [==============================] - 1s 1s/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  0
1/1 [==============================] - 1s 501ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  6
1/1 [==============================] - 0s 483ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  4
1/1 [==============================] - 1s 931ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  1
1/1 [==============================] - 0s 494ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  7
1/1 [==============================] - 2s 2s/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  9
1/1 [==============================] - 1s 967ms/step - loss: 2.2770 - sparse_categorical_accuracy: 0.7000
Client No  5
1/1 [===========================

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 835ms/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  3
1/1 [==============================] - 1s 1s/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  8
1/1 [==============================] - 0s 499ms/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  2
1/1 [==============================] - 2s 2s/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  7
1/1 [==============================] - 1s 1s/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  1
1/1 [==============================] - 0s 494ms/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  7
1/1 [==============================] - 1s 953ms/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  4
1/1 [==============================] - 1s 938ms/step - loss: 2.2721 - sparse_categorical_accuracy: 0.7000
Client No  2
1/1 [==============================] - 1s 1s/step - loss:

...

1/1 [==============================] - 0s 469ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  1
1/1 [==============================] - 0s 471ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  5
1/1 [==============================] - 1s 919ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  7
1/1 [==============================] - 1s 940ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  4
1/1 [==============================] - 1s 837ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  4
1/1 [==============================] - 1s 793ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  8
1/1 [==============================] - 1s 826ms/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  3
1/1 [==============================] - 2s 2s/step - loss: 0.0115 - sparse_categorical_accuracy: 1.0000
Client No  0
1/1 [==============================] - 1s 818ms/ste

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 784ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  2
1/1 [==============================] - 1s 786ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  3
1/1 [==============================] - 1s 964ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  1
1/1 [==============================] - 1s 841ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  5
1/1 [==============================] - 1s 787ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  5
1/1 [==============================] - 1s 907ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  9
1/1 [==============================] - 1s 793ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  5
1/1 [==============================] - 1s 950ms/step - loss: 0.0085 - sparse_categorical_accuracy: 1.0000
Client No  1
1/1 [==============================] - 0s 443ms/

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


round  34
Client No  1
1/1 [==============================] - 0s 466ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  6
1/1 [==============================] - 1s 773ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  8
1/1 [==============================] - 1s 1s/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  2
1/1 [==============================] - 1s 799ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  9
1/1 [==============================] - 1s 905ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  6
1/1 [==============================] - 1s 917ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  8
1/1 [==============================] - 0s 466ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  2
1/1 [==============================] - 1s 943ms/step - loss: 0.0066 - sparse_categorical_accuracy: 1.0000
Client No  0
1/1 [=======================